In [39]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np

# read_csv() 함수로 df 생성
df = pd.read_csv('./data/auto-mpg.csv', header=None)

# 열 이름을 지정
df.columns = ['mpg','cylinders','displacement','horsepower','weight',
              'acceleration','model year','origin','name'] 

# horsepower 열의 누락 데이터('?') 삭제하고 실수형으로 변환
df['horsepower'] = df['horsepower'].replace('?', np.nan)      # '?'을 np.nan으로 변경
df = df.dropna(subset=['horsepower'], axis=0)                 # 누락데이터 행을 삭제
df['horsepower'] = df['horsepower'].astype('float')           # 문자열을 실수형으로 변환

# np.histogram 으로 3개의 bin으로 나누는 경계 값의 리스트 구하기
count, bin_dividers = np.histogram(df['horsepower'], bins=3)

# 3개의 bin에 이름 지정
bin_names = ['저출력', '보통출력', '고출력']

# pd.cut 으로 각 데이터를 3개의 bin에 할당
df['hp_bin'] = pd.cut(x=df['horsepower'],     # 데이터 배열
                      bins=bin_dividers,      # 경계 값 리스트
                      labels=bin_names,       # bin 이름
                      include_lowest=True)    # 첫 경계값 포함

# sklern 라이브러리 불러오기
from sklearn import preprocessing    

# 전처리를 위한 encoder 객체 만들기
label_encoder = preprocessing.LabelEncoder()       # label encoder 생성
onehot_encoder = preprocessing.OneHotEncoder()     # one hot encoder 생성

# label encoder로 문자열 범주를 숫자형 범주로 변환
onehot_labeled = label_encoder.fit_transform(df['hp_bin'].head(15))  
print(onehot_labeled)
print(type(onehot_labeled))

# 2차원 행렬로 형태 변경
onehot_reshaped = onehot_labeled.reshape(len(onehot_labeled), 1) 
print(onehot_reshaped)
print(type(onehot_reshaped))

# 희소행렬로 변환
onehot_fitted = onehot_encoder.fit_transform(onehot_reshaped)
print(onehot_fitted)
print(type(onehot_fitted))

[1 1 1 1 1 0 0 0 0 0 0 1 1 0 2]
<class 'numpy.ndarray'>
[[1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [2]]
<class 'numpy.ndarray'>
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 15 stored elements and shape (15, 3)>
  Coords	Values
  (0, 1)	1.0
  (1, 1)	1.0
  (2, 1)	1.0
  (3, 1)	1.0
  (4, 1)	1.0
  (5, 0)	1.0
  (6, 0)	1.0
  (7, 0)	1.0
  (8, 0)	1.0
  (9, 0)	1.0
  (10, 0)	1.0
  (11, 1)	1.0
  (12, 1)	1.0
  (13, 0)	1.0
  (14, 2)	1.0
<class 'scipy.sparse._csr.csr_matrix'>


In [40]:
%pip install pandas numpy

Note: you may need to restart the kernel to use updated packages.


In [41]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np

# read_csv() 함수로 df 생성
df = pd.read_csv('./data/auto-mpg.csv', header=None)

# 열 이름을 지정
df.columns = ['mpg','cylinders','displacement','horsepower','weight',
              'acceleration','model year','origin','name'] 
df.head(2)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320


In [42]:
# 연비를 3개 구간으로 나눠서 새로운 컬럼명으로 추가
# 저연비, 보통, 고연비
# 연속형(수치형) -> 범주형(category)형 분리
df.mpg
df['mpg_qcut'] =  pd.qcut(df.mpg,q=3,labels=['저연비', '보통', '고연비'])
print(df.mpg_qcut.value_counts())
# 계급구간확인  라벨없이 적용후
temp = pd.qcut(df.mpg,q=3)
print(f'구간별 데이터 범위 : {temp.cat.categories}')

mpg_qcut
저연비    143
고연비    133
보통     122
Name: count, dtype: int64
구간별 데이터 범위 : IntervalIndex([(8.999, 19.0], (19.0, 26.933], (26.933, 46.6]], dtype='interval[float64, right]')


In [43]:
size, bins_range =  np.histogram(df.mpg,bins=np.array(3))
df['mpg_cut'] = pd.cut(df.mpg,bins=bins_range,labels=['저연비', '보통', '고연비'])
df.mpg_cut.value_counts()
print(f'계급별 구간 : {bins_range}')

계급별 구간 : [ 9.         21.53333333 34.06666667 46.6       ]


In [44]:
pd.get_dummies(df.mpg_qcut,dtype=float)

,저연비,보통,고연비
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0
...,...,...,...
393,0.0,0.0,1.0
394,0.0,0.0,1.0
395,0.0,0.0,1.0
396,0.0,0.0,1.0


In [45]:
pd.get_dummies(df.mpg_cut,dtype=float)

,저연비,보통,고연비
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0
...,...,...,...
393,0.0,1.0,0.0
394,0.0,0.0,1.0
395,0.0,1.0,0.0
396,0.0,1.0,0.0


In [46]:
# 다중 공선성 : 회귀분석(여러변수로 결과를 예측하는 모델), 독립변수(컬럼들..)끼리 서로 너무 강하게 상관관계를 가질때 문제가 생김
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [47]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

# read_csv() 함수로 df 생성
df = pd.read_csv('./data/auto-mpg.csv', header=None)
df.columns = ['mpg','cylinders','displacement','horsepower','weight',
              'acceleration','model year','origin','name'] 
# 범주형 데이터로 변환
df['mpg_qcut'] =  pd.qcut(df.mpg,q=3,labels=['저연비', '보통', '고연비'])

encoder = OneHotEncoder(sparse_output=False)  #객체
# sklearn 계열은 사용방법이 통일   fit : 적용  transform : 변환    fit_transform은 두개를 한꺼번에 실행
temp = encoder.fit_transform(df[['mpg_qcut']])  # 2차원데이터가 와야 함
cols = encoder.get_feature_names_out(['mpg_qcut'])
pd.DataFrame(temp, columns=cols)
pd.concat([df.drop(columns=['mpg_qcut']), pd.DataFrame(temp, columns=cols)],axis=1)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,mpg_qcut_고연비,mpg_qcut_보통,mpg_qcut_저연비
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu,0.0,0.0,1.0
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,0.0,0.0,1.0
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite,0.0,0.0,1.0
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst,0.0,0.0,1.0
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.00,2790.0,15.6,82,1,ford mustang gl,1.0,0.0,0.0
394,44.0,4,97.0,52.00,2130.0,24.6,82,2,vw pickup,1.0,0.0,0.0
395,32.0,4,135.0,84.00,2295.0,11.6,82,1,dodge rampage,1.0,0.0,0.0
396,28.0,4,120.0,79.00,2625.0,18.6,82,1,ford ranger,1.0,0.0,0.0


In [48]:
encoder2 = OneHotEncoder(sparse_output=True)
encoder2.fit_transform(df[['mpg_qcut']])

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 398 stored elements and shape (398, 3)>

In [49]:
encoder2 = OneHotEncoder(sparse_output=False)
encoder2.fit_transform(df[['mpg_qcut']])

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], shape=(398, 3))

In [50]:
# get_dummy()  : 빠르게 확인, 컬럼명 유지   pandas Dataframe / Series
    #  데이터 탐색, 시각화 ,작은데이터셋

# OneHotEncoder()  numpy array, DataFrame 
    # 머신러닝 파이라인/모델 학습
    #fit , transform --> 학습-예측데이터 일관성 유지, 큰데이터셋

In [51]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

# read_csv() 함수로 df 생성
df = pd.read_csv('./data/auto-mpg.csv', header=None)
df.columns = ['mpg','cylinders','displacement','horsepower','weight',
              'acceleration','model year','origin','name'] 
# 범주형 데이터로 변환
df['mpg_qcut'] =  pd.qcut(df.mpg,q=3,labels=['저연비', '보통', '고연비'])

encoder = OneHotEncoder(sparse_output=True)  #객체
# sklearn 계열은 사용방법이 통일   fit : 적용  transform : 변환    fit_transform은 두개를 한꺼번에 실행
temp = encoder.fit_transform(df[['mpg_qcut']])  # 2차원데이터가 와야 함
cols = encoder.get_feature_names_out(['mpg_qcut'])
pd.DataFrame.sparse.from_spmatrix(temp, columns=cols)
pd.concat([df.drop(columns=['mpg_qcut']), pd.DataFrame.sparse.from_spmatrix(temp, columns=cols)],axis=1)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,mpg_qcut_고연비,mpg_qcut_보통,mpg_qcut_저연비
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu,0,0,1.0
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,0,0,1.0
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite,0,0,1.0
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst,0,0,1.0
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.00,2790.0,15.6,82,1,ford mustang gl,1.0,0,0
394,44.0,4,97.0,52.00,2130.0,24.6,82,2,vw pickup,1.0,0,0
395,32.0,4,135.0,84.00,2295.0,11.6,82,1,dodge rampage,1.0,0,0
396,28.0,4,120.0,79.00,2625.0,18.6,82,1,ford ranger,1.0,0,0


In [52]:
train_df = df.drop(columns=['mpg'])
train_df.head(2)
# cylinders,model year, origin,mpg_qcut --> onehot
# onehot 이후에 onehot에 대상이된 컬럼은 drop
# 제조사컬럼에서 제조사만 추출해서 -> onehot
# 하나의 데이터프레임으로 결합 concat

,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,mpg_qcut
0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu,저연비
1,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,저연비


In [53]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
origin_cols = ['cylinders','model year', 'origin','maker']
train_df['maker'] = [n_list[0] for n_list in df['name'].str.split()]
total_onehots = []
for colname in origin_cols:
    encoder_fit_transform = encoder.fit_transform(train_df[[colname]])
    cols = encoder.get_feature_names_out([colname])
    total_onehots.append(pd.DataFrame.sparse.from_spmatrix(encoder_fit_transform,columns=cols))

In [54]:
total_onehots.insert(0,train_df)
new_train_df = pd.concat(total_onehots,axis=1)
new_train_df = new_train_df.drop(columns=origin_cols)
new_train_df.head()

,displacement,horsepower,weight,acceleration,name,mpg_qcut,cylinders_3,cylinders_4,cylinders_5,cylinders_6,...,maker_renault,maker_saab,maker_subaru,maker_toyota,maker_toyouta,maker_triumph,maker_vokswagen,maker_volkswagen,maker_volvo,maker_vw
0,307.0,130.0,3504.0,12.0,chevrolet chevelle malibu,저연비,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,350.0,165.0,3693.0,11.5,buick skylark 320,저연비,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,318.0,150.0,3436.0,11.0,plymouth satellite,저연비,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,304.0,150.0,3433.0,12.0,amc rebel sst,저연비,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,302.0,140.0,3449.0,10.5,ford torino,저연비,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
# 범주형 데이터... 범위내에서 결정되는 데이터  -category
# 연속형 데이터 : 범위가 없는 변화무쌍한 데이터
# 개 고양이 새 ->범주형데이터  0,과 1로 표현
# 고양이  [1,0,0] 고양이on 나머지 off
# 새 [0,1,0]
# 개 [0,0,1]
# 고양이 0, 새 1, 개 2 --> ?

# 키를 예측 모델  , 신발사이즈 변수사용  1(작다),2(중간),3(크다) --> 큰신발을 신은 사람은 키가 크다 
# 3번신발이 신은 사람이 1번신발을 신은 사람보다   신발사이즈가 3배증가 --> 아. 키도 3개크다고 인식
# 신발사이즈 변수 : 카테고리... 모델학습할때. 참고용도
# 1,0,0  0,1,0 0,0,1

In [60]:
from sklearn.linear_model import LinearRegression  # 선형 회귀2
# 작은 : 0 보통 1 크 2
data = {
    'size' : [0,0,0,1,1,1,2,2,2],
    'height':[120,121,119,122,123,121,125,124,126]
}
df = pd.DataFrame(data)
# 실제평균
df.groupby('size')['height'].mean()

size
0    120.0
1    122.0
2    125.0
Name: height, dtype: float64

In [70]:
model_lr = LinearRegression()
X = df.drop(columns=['height']).to_numpy()
y = df['height']
X.shape , y.shape, type(y)
model_ly = LinearRegression()
model_ly.fit(X,y)
predicted_y0 = model_ly.predict([[0]])[0]
predicted_y1 = model_ly.predict([[1]])[0]
predicted_y2 = model_ly.predict([[2]])[0]
predicted_y0,predicted_y1,predicted_y2

(np.float64(119.83333333333333),
 np.float64(122.33333333333333),
 np.float64(124.83333333333333))

In [73]:
# ont hot을 적용해서 학습하고 예측
df_encoded = pd.get_dummies(df,columns=['size'])
X_onehot = df_encoded[['size_0',	'size_1']].to_numpy()
model_onehot = LinearRegression()
model_onehot.fit(X_onehot,y)
predicted_onehot_y0 = model_onehot.predict([[1,0]])[0]
predicted_onehot_y1 = model_onehot.predict([[0,1]])[0]
predicted_onehot_y2 = model_onehot.predict([[0,0]])[0]
predicted_onehot_y0,predicted_onehot_y1,predicted_onehot_y2

(np.float64(120.0), np.float64(122.0), np.float64(125.0))